In [ ]:
import tensorflow as tf

MODEL_DIR='../results/models/'
MODEL_NAME='Model_1'

In [ ]:
tf.keras.backend.clear_session()
model=tf.saved_model.load(os.path.join(os.path.join(MODEL_DIR, 'output'), 'MODEL_NAME'))

In [ ]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import gc
import glob
import io
import IPython
import json
import numpy as np
import pathlib
import pandas as pd
import sys
import cv2
import math
from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt

from PIL import Image
import tensorflow as tf

INPUT_DIR = '/kaggle/input/tensorflow-great-barrier-reef/'
sys.path.insert(0, INPUT_DIR)
import greatbarrierreef

In [ ]:
print(tf.__version__)
print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))

In [ ]:
tf.keras.backend.clear_session()
detect_fn_tf_odt = tf.saved_model.load(os.path.join(os.path.join(MODEL_DIR, 'output'), 'saved_model'))

In [ ]:


import greatbarrierreef
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test() 

In [ ]:
DETECTION_THRESHOLD = 0.3

submission_dict = {
    'id': [],
    'prediction_string': [],
}

for (image_np, sample_prediction_df) in iter_test:
    height, width, _ = image_np.shape
    
    # Run object detection using the TensorFlow model.
    detections = detect(image_np)
    
    # Parse the detection result and generate a prediction string.
    num_detections = detections['num_detections'][0].numpy().astype(np.int32)
    predictions = []
    for index in range(num_detections):
        score = detections['detection_scores'][0][index].numpy()
        if score < DETECTION_THRESHOLD:
            continue

        bbox = detections['detection_boxes'][0][index].numpy()
        y_min = int(bbox[0] * height)
        x_min = int(bbox[1] * width)
        y_max = int(bbox[2] * height)
        x_max = int(bbox[3] * width)
        
        bbox_width = x_max - x_min
        bbox_height = y_max - y_min
        
        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    
    # Generate the submission data.
    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)

    print('Prediction:', prediction_str)

In [ ]:
!ls {folders["CHECKPOINT_PATH"]}/ckpt-*.index

In [ ]:


configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

ckps = glob.glob(folders["CHECKPOINT_PATH"]+"/ckpt-*.index")
ckps.sort(key=os.path.getmtime)

ckp_file = ckps[-1][:-6]
print(ckp_file)
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(ckp_file).expect_partial()

def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections



In [ ]:

def get_image_pred(df, idx):

    image_np = load_image_into_np(df.iloc[idx].image_path)
    
    height, width, _ = image_np.shape
    input_tensor = tf.cast(np.expand_dims(image_np, 0), tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = detections['num_detections'][0].numpy().astype(np.int32) 
    bboxes = []
    scores = []
    classes = []
    DETECTION_THRESHOLD = 0.15
    
    for i in range(num_detections):
        score = detections['detection_scores'][0][i].numpy()
        
        if score < DETECTION_THRESHOLD:
            continue

        bboxes.append(list(detections['detection_boxes'][0][i].numpy()))
        scores.append(score)
        classes.append(1)
    img = plot_detections(image_np, np.array(bboxes), np.array(classes), np.array(scores), category_index, unc=True)
    return img

cnt = 10
fig, ax = plt.subplots(cnt, 2, figsize = (20,40))
for row in range(cnt):
    idx = np.random.randint(0,val_data_df.shape[0])
    gt = get_image_with_annotation(val_data_df, idx)
    pred = get_image_pred(val_data_df, idx)
    
    ax[row][0].imshow(gt)
    ax[row][0].set_xticks([])
    ax[row][0].set_yticks([])
    ax[row][0].set_title(f"GT_{idx}")
    
    ax[row][1].imshow(pred)
    ax[row][1].set_xticks([])
    ax[row][1].set_yticks([])
    ax[row][1].set_title(f"PR_{idx}")
plt.tight_layout()
plt.show()



# Submission

In [ ]:
env = greatbarrierreef.make_env()  
iter_test = env.iter_test() 

In [ ]:
DETECTION_THRESHOLD = 0.15

for (image_np, df) in iter_test:
    height, width, _ = image_np.shape
    
    input_tensor = tf.cast(np.expand_dims(image_np, 0), tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = detections['num_detections'][0].numpy().astype(np.int32)
    predictions = []
    
    for index in range(num_detections):
        score = detections['detection_scores'][0][index].numpy()

        if score < DETECTION_THRESHOLD:
            continue

        bbox = detections['detection_boxes'][0][index].numpy()
        y_min = int(bbox[0] * height)
        x_min = int(bbox[1] * width)
        y_max = int(bbox[2] * height)
        x_max = int(bbox[3] * width)

        bbox_width = x_max - x_min
        bbox_height = y_max - y_min

        predictions.append(f'{score:.2f} {x_min} {y_min} {bbox_width} {bbox_height}')
        
        
    prediction_str = ' '.join(predictions)
    df['annotations'] = prediction_str
    env.predict(df)

sub_df = pd.read_csv('submission.csv')
sub_df.head()

In [ ]:
!rm -rf ./api
!rm -rf ./data
!rm -rf ./pre-trained-models